In [2]:
import numpy as np
import PIL
import PIL.Image
import tensorflow as tf
import pathlib
from os import listdir
from random import shuffle
import cv2
from tensorflow.keras import layers

KeyboardInterrupt: 

In [2]:

#dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
archive = './train/isolated_alphabets_per_alphabet'
data_dir = pathlib.Path(archive).with_suffix('')

In [3]:
training_data = []
ok = False
categories = ['ain_begin' , 'ain_end' , 'ain_middle' , 'ain_regular','alif_end' , 'alif_hamza' , 'alif_regular','beh_begin' , 'beh_end' , 'beh_middle' , 'beh_regular','dal_end' , 'dal_regular','feh_begin' , 'feh_end' , 'feh_middle' , 'feh_regular','heh_begin' , 'heh_end' , 'heh_middle' , 'heh_regular','jeem_begin' , 'jeem_end' , 'jeem_middle' , 'jeem_regular','kaf_begin' , 'kaf_end' , 'kaf_middle' , 'kaf_regular','lam_alif','lam_begin' , 'lam_end' , 'lam_middle' , 'lam_regular','meem_begin' , 'meem_end' , 'meem_middle' , 'meem_regular','noon_begin' , 'noon_end' , 'noon_middle' , 'noon_regular','qaf_begin' , 'qaf_end' , 'qaf_middle' , 'qaf_regular','raa_end' , 'raa_regular','ain_begin' , 'ain_end' , 'ain_regular','sad_begin' , 'sad_end' , 'sad_middle' , 'sad_regular','seen_begin' , 'seen_end' , 'seen_middle' , 'seen_regular','tah_end' , 'tah_middle' , 'tah_regular','waw_end' , 'waw_regular','yaa_begin' , 'yaa_end' , 'yaa_middle' , 'yaa_regular']
test_directory = ""
train_directory = ""
directory = './train/isolated_alphabets_per_alphabet'
def training_data_initializer():
    counter = 0
    for category in categories:
        path = directory+'/'+category
        Image_category = categories.index(category)
        for img in listdir(path):
            try:
                counter+=1
                if counter % 1000 == 0:
                    print(counter)
                    if  counter  == 2015:
                        print(training_data[i])
                img_array = cv2.imread(f'{path}/{img}', cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array , (32 , 32))
                training_data.append([new_array , Image_category])
            except Exception as e:
                pass
training_data_initializer()

shuffle(training_data)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000


In [5]:
X = []
Y = []
for features , label in training_data:
    X.append(features)
    Y.append(label)
X = np.array(X).reshape(-1 , 32 , 32, 1)

In [120]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(X , Y , test_size = 0.2 , random_state = 0)
x_train = tf.stack(x_train)
y_train = tf.convert_to_tensor(y_train)
x_test = tf.stack(x_test)
y_test = tf.convert_to_tensor(y_test)

In [121]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).prefetch(16)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32).prefetch(16)

In [128]:
#I like to seperate them apart as independent layers
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=(32 , 32 , 1)))
model.add(tf.keras.layers.AveragePooling2D())

model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.AveragePooling2D())

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(units=120, activation='relu')) #120

model.add(tf.keras.layers.Dense(units=84, activation='relu')) #84
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(units=68, activation = 'softmax'))


In [90]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_31 (Conv2D)          (None, 30, 30, 6)         60        
                                                                 
 average_pooling2d_27 (Aver  (None, 15, 15, 6)         0         
 agePooling2D)                                                   
                                                                 
 conv2d_32 (Conv2D)          (None, 13, 13, 16)        880       
                                                                 
 average_pooling2d_28 (Aver  (None, 6, 6, 16)          0         
 agePooling2D)                                                   
                                                                 
 flatten_8 (Flatten)         (None, 576)               0         
                                                                 
 dense_21 (Dense)            (None, 256)             

In [129]:
from tensorflow.keras.utils import to_categorical
opt = tf.keras.optimizers.Adam(learning_rate=0.0005) #0.0001 DEFAULT
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer=opt, metrics=['accuracy'])
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=1,
    min_lr=0.00001
)

In [81]:
score = model.evaluate(x_test,y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

132/132 [==============================] - 1s 8ms/step - loss: 2.6936 - accuracy: 0.2889
Test loss: 2.693567991256714
Test accuracy: 0.2888730466365814


In [130]:
model.fit(train_ds, 
          validation_data= test_ds, 
          epochs = 80,
          callbacks=[reduce_lr])

Epoch 1/80


921/921 [==============================] - 19s 19ms/step - loss: 4.5616 - accuracy: 0.0279 - val_loss: 4.1969 - val_accuracy: 0.0301 - lr: 5.0000e-04
Epoch 2/80
921/921 [==============================] - 20s 22ms/step - loss: 4.1907 - accuracy: 0.0284 - val_loss: 4.1834 - val_accuracy: 0.0309 - lr: 5.0000e-04
Epoch 3/80
780/921 [========================>.....] - ETA: 2s - loss: 4.1836 - accuracy: 0.0281

KeyboardInterrupt: 

In [13]:
import pandas as pd
df = pd.read_csv('Sample Submission File.csv')

In [91]:
prediction = model.predict(x_test)
y_predict = ''
for i in range(prediction.shape[0]):
    y_predict=np.argmax(prediction[i])

263/263 [==============================] - 2s 7ms/step
feh_middle
dal_regular
lam_end
lam_end
sad_regular
heh_regular
waw_regular
tah_middle
sad_regular
sad_begin
tah_end
ain_end
qaf_end
meem_begin
seen_begin
kaf_begin
heh_middle
alif_end
alif_regular
lam_begin
meem_begin
noon_end
sad_middle
kaf_end
heh_begin
kaf_end
ain_end
feh_middle
noon_regular
dal_end
seen_middle
qaf_regular
sad_regular
heh_regular
tah_middle
noon_regular
dal_regular
ain_regular
noon_middle
lam_end
seen_begin
dal_regular
yaa_begin
feh_begin
raa_end
ain_middle
jeem_end
dal_end
ain_begin
yaa_middle
lam_end
jeem_end
heh_regular
ain_regular
alif_hamza
lam_begin
ain_begin
heh_begin
seen_begin
kaf_regular
ain_begin
feh_regular
noon_regular
meem_regular
feh_end
ain_regular
seen_begin
feh_regular
lam_alif
seen_middle
beh_end
tah_middle
kaf_end
noon_begin
meem_begin
meem_regular
ain_begin
ain_begin
beh_begin
seen_end
kaf_begin
kaf_middle
ain_regular
qaf_end
meem_begin
jeem_middle
meem_regular
feh_regular
ain_begin
ain_regu

In [17]:
cnt = 0
prediction_data = []
for category in categories:
        path = './test/test/'+category

        for img in listdir(path):
                img_array = cv2.imread(f'{path}/{img}' , cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array , (32 , 32))
                prediction_data.append(new_array)
prediction_data = np.array(prediction_data).reshape(-1 , 32 , 32 ,1)
prediction_data.shape

(13600, 32, 32, 1)

In [14]:
def suffix_decision(suffix):                              
    if suffix.endswith('end'):
        return suffix.replace('_end','')
    elif suffix.endswith('begin'):
        return suffix.replace('_begin','')
    elif suffix.endswith('regular'):
        return suffix.replace('_regular','')
    elif suffix.endswith('middle'):
        return suffix.replace('_middle' , '')
    elif suffix.endswith('hamza'):
        return suffix.replace('_hamza' , '')
    else:
        return suffix

In [32]:
prediction = model.predict(prediction_data)
y_predict = 0
long_letter = ""
desired_letter = ""
for i in range(13000):
    y_predict=np.argmax(prediction[i])
    long_letter = categories[y_predict]
    desired_letter = suffix_decision(long_letter)
    df.loc[i , 'Letter'] = desired_letter
    #if(df.loc[1 , 'ID'] == None):
df.to_csv("Sample Submission File.csv" , index = False)

425/425 [==============================] - 2s 5ms/step


In [52]:

score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

132/132 [==============================] - 2s 8ms/step - loss: 2.2842 - accuracy: 0.6362
Test loss: 2.2841575145721436
Test accuracy: 0.636233925819397
